# Simple RAG Pipeline

In [1]:
## Import getpass to create propmt to add env variables

import getpass
import os
from dotenv import load_dotenv
load_dotenv()

True

## Loaders

### Text Loader

In [2]:
## Import text loader 
from langchain_community.document_loaders import TextLoader

loader = TextLoader('docs\Maharana_PratapJi.txt')

text = loader.load()

text

[Document(metadata={'source': 'docs\\Maharana_PratapJi.txt'}, page_content="Source : https://www.geeksforgeeks.org/maharana-pratap-biography/\n\n\nMaharana Pratap was a brave Rajput king in India who stood up against the powerful Mughal Empire. He is famous for his courage and refusal to surrender, making him a symbol of resistance and pride for many.\n\nMaharana Pratap was a Hindu Rajput king who ruled the Mewar region for 35 years. He was the 13th Maharana of Mewar and a member of the Sisodia dynasty.\n\nIn this article, we have covered the complete Maharana Pratap Biography, Why he is Famous, His Family tree, How did he died. His battles and many more in Detail.\n\nLetâ€™s dive right in.\n\nMaharana Pratap Life Overview\nThe following is the Life Overview of Maharana Pratap:\n\nAspect\t-->  Details\nFull Name\t--> Maharana Pratap Singh\nBirth Date\t--> May 9, 1540\nBirthplace\t--> Kumbhalgarh, Mewar (present-day Rajasthan)\nReign\t-->   1572â€“1597(25 years)\nDynasty\t--> Sisodia Ra

In [3]:
text[0].page_content

"Source : https://www.geeksforgeeks.org/maharana-pratap-biography/\n\n\nMaharana Pratap was a brave Rajput king in India who stood up against the powerful Mughal Empire. He is famous for his courage and refusal to surrender, making him a symbol of resistance and pride for many.\n\nMaharana Pratap was a Hindu Rajput king who ruled the Mewar region for 35 years. He was the 13th Maharana of Mewar and a member of the Sisodia dynasty.\n\nIn this article, we have covered the complete Maharana Pratap Biography, Why he is Famous, His Family tree, How did he died. His battles and many more in Detail.\n\nLetâ€™s dive right in.\n\nMaharana Pratap Life Overview\nThe following is the Life Overview of Maharana Pratap:\n\nAspect\t-->  Details\nFull Name\t--> Maharana Pratap Singh\nBirth Date\t--> May 9, 1540\nBirthplace\t--> Kumbhalgarh, Mewar (present-day Rajasthan)\nReign\t-->   1572â€“1597(25 years)\nDynasty\t--> Sisodia Rajput\nFather\t--> Maharana Udai Singh II\nMother\t--> Rani Jeevant Kanwa\nS

### PDF  Loader 

In [5]:
from langchain_community.document_loaders import PyPDFLoader

pdfloader = PyPDFLoader('docs/attention.pdf')

pdf_text = pdfloader.load()

len(pdf_text)

15

In [6]:
pdf_text[2].json()

'{"id": null, "metadata": {"source": "docs/attention.pdf", "page": 2}, "page_content": "Figure 1: The Transformer - model architecture.\\nThe Transformer follows this overall architecture using stacked self-attention and point-wise, fully\\nconnected layers for both the encoder and decoder, shown in the left and right halves of Figure 1,\\nrespectively.\\n3.1 Encoder and Decoder Stacks\\nEncoder: The encoder is composed of a stack of N= 6 identical layers. Each layer has two\\nsub-layers. The first is a multi-head self-attention mechanism, and the second is a simple, position-\\nwise fully connected feed-forward network. We employ a residual connection [ 11] around each of\\nthe two sub-layers, followed by layer normalization [ 1]. That is, the output of each sub-layer is\\nLayerNorm( x+ Sublayer( x)), where Sublayer( x)is the function implemented by the sub-layer\\nitself. To facilitate these residual connections, all sub-layers in the model, as well as the embedding\\nlayers, produce

## Vector Store and Retrival

In [7]:
## RecursiveCharacterTextSplitter will split test into small chunks due to llm has less context size.

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter( chunk_size=512,chunk_overlap=50)

# Conveting TXT text to chunk
# chunk_text = text_splitter.split_documents(text)
# chunk_text[:5]

# Conveting PDF text to chunk
chunk_pdf = text_splitter.split_documents(pdf_text)
chunk_pdf[:5]

[Document(metadata={'source': 'docs/attention.pdf', 'page': 0}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗'),
 Document(metadata={'source': 'docs/attention.pdf', 'page': 0}, page_content='aidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect 

In [8]:
chunk_pdf

[Document(metadata={'source': 'docs/attention.pdf', 'page': 0}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗'),
 Document(metadata={'source': 'docs/attention.pdf', 'page': 0}, page_content='aidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect 

### Adding Cohere Embedder

In [9]:
## we have chunks, we need to ADD those chunct into vector database so we need to have vector embedding first then we can insert the record into cromadb.

if not os.getenv("COHERE_API_KEY"):
    os.environ["COHERE_API_KEY"] = getpass.getpass()

In [10]:
from langchain_cohere import CohereEmbeddings

embeddings = CohereEmbeddings(
    model="embed-english-v3.0",
)

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\GAURAV\AppData\Local\sagemaker\sagemaker\config.yaml


In [ ]:
import xyz

### ChromaDB

In [11]:
from langchain_chroma import Chroma

vector_store = Chroma(
    chunk_pdf,
    embedding_function=embeddings,
)

ValueError: Expected collection name that (1) contains 3-63 characters, (2) starts and ends with an alphanumeric character, (3) otherwise contains only alphanumeric characters, underscores or hyphens (-), (4) contains no two consecutive periods (..) and (5) is not a valid IPv4 address, got [Document(metadata={'source': 'docs/attention.pdf', 'page': 0}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗'), Document(metadata={'source': 'docs/attention.pdf', 'page': 0}, page_content='aidan@cs.toronto.eduŁukasz Kais

In [ ]:
from uuid import uuid4

# uuids = [str(uuid4()) for _ in range(len(chunk_text))] + [str(uuid4()) for _ in range(len(chunk_pdf))] 
# uuids = [str(uuid4()) for _ in range(len(chunk_pdf))] 
# vector_store.add_documents(documents=chunk_text, ids=uuids)
vector_store.add_documents(documents=chunk_pdf, ids=uuids)

### Retrieval

In [ ]:
results = vector_store.similarity_search(
    "What was the last war of Maharana Pratap?",
    k=2
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

In [ ]:
results = vector_store.similarity_search(
    "What Fixed Income Instruments?",
    k=2
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")